In [2]:
# --- Step 1: Upload File ---
from google.colab import files
import pandas as pd
import sqlite3

print("📂 Please upload Online Retail.xlsx")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# --- Step 2: Load & Clean Data ---
df = pd.read_excel(file_name)
df = df.dropna(subset=['CustomerID'])
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Revenue'] = df['Quantity'] * df['UnitPrice']

# --- Step 3: Create SQLite DB ---
conn = sqlite3.connect(":memory:")
df.to_sql("transactions", conn, index=False, if_exists="replace")

# --- Step 4: Segmentation Query ---
segmentation = pd.read_sql("""
WITH CustAgg AS (
    SELECT
        CustomerID,
        SUM(Revenue) AS TotalRevenue,
        COUNT(DISTINCT InvoiceNo) AS Frequency
    FROM transactions
    GROUP BY CustomerID
)
SELECT
    CustomerID,
    TotalRevenue,
    Frequency,
    CASE
        WHEN TotalRevenue >= (SELECT TotalRevenue FROM CustAgg ORDER BY TotalRevenue DESC LIMIT (SELECT COUNT(*)/10 FROM CustAgg))
            THEN 'High Value'
        WHEN Frequency >= (SELECT Frequency FROM CustAgg ORDER BY Frequency DESC LIMIT (SELECT COUNT(*)/4 FROM CustAgg))
            THEN 'Frequent Low Spender'
        ELSE 'Other'
    END AS Segment
FROM CustAgg
""", conn)

# --- Step 5: Monthly Trends Query ---
monthly = pd.read_sql("""
SELECT
    strftime('%Y-%m', InvoiceDate) AS Month,
    SUM(Revenue) AS MonthlyRevenue,
    COUNT(DISTINCT CustomerID) AS UniqueCustomers
FROM transactions
GROUP BY strftime('%Y-%m', InvoiceDate)
ORDER BY Month
""", conn)

# --- Step 6: Customer Counts & Revenue per Segment ---
segment_counts = segmentation.groupby("Segment")["CustomerID"].nunique().reset_index()
segment_counts.rename(columns={"CustomerID": "CustomerCount"}, inplace=True)

segment_revenue = segmentation.groupby("Segment")["TotalRevenue"].sum().reset_index()

# Merge into combined summary
combined_summary = pd.merge(segment_counts, segment_revenue, on="Segment")

# Add % Revenue Share
total_revenue = combined_summary["TotalRevenue"].sum()
combined_summary["RevenueShare(%)"] = round(combined_summary["TotalRevenue"] / total_revenue * 100, 2)

# --- Step 7: Pareto Analysis (Top 10% Customers Contribution) ---
total_customers = segmentation.shape[0]
cutoff = int(0.10 * total_customers)

pareto = segmentation.sort_values("TotalRevenue", ascending=False).head(cutoff)
top10_revenue = pareto["TotalRevenue"].sum()

pareto_summary = pd.DataFrame({
    "Metric": ["Total Customers", "Top 10% Customers", "Total Revenue", "Top 10% Revenue", "Top 10% Revenue Share (%)"],
    "Value": [total_customers, cutoff, total_revenue, top10_revenue, round(top10_revenue / total_revenue * 100, 2)]
})

# --- Step 8: Save All Outputs ---
segmentation.to_csv("Customer_Segmentation.csv", index=False)
monthly.to_csv("Monthly_Trends.csv", index=False)
segment_counts.to_csv("Segment_Counts.csv", index=False)
segment_revenue.to_csv("Segment_Revenue.csv", index=False)
combined_summary.to_csv("Segment_Summary.csv", index=False)
pareto_summary.to_csv("Pareto_Summary.csv", index=False)



📂 Please upload Online Retail.xlsx


Saving Online Retail.xlsx to Online Retail.xlsx


In [3]:
# --- Fix Segmentation in Python instead of SQL ---
cust_agg = df.groupby("CustomerID").agg(
    TotalRevenue=("Revenue", "sum"),
    Frequency=("InvoiceNo", "nunique")
).reset_index()

# thresholds
rev_threshold = cust_agg["TotalRevenue"].quantile(0.90)   # top 10% revenue
freq_threshold = cust_agg["Frequency"].quantile(0.75)     # top 25% frequency

# assign segment
def assign_segment(row):
    if row["TotalRevenue"] >= rev_threshold:
        return "High Value"
    elif row["Frequency"] >= freq_threshold:
        return "Frequent Low Spender"
    else:
        return "Other"

cust_agg["Segment"] = cust_agg.apply(assign_segment, axis=1)

# now cust_agg is your segmentation DataFrame
segmentation = cust_agg


In [4]:
combined_summary = (
    segmentation.groupby("Segment")
    .agg(
        CustomerCount=("CustomerID", "nunique"),
        TotalRevenue=("TotalRevenue", "sum")
    )
    .reset_index()
)

total_revenue = combined_summary["TotalRevenue"].sum()
combined_summary["RevenueShare(%)"] = round(
    combined_summary["TotalRevenue"] / total_revenue * 100, 2
)

print(combined_summary)


                Segment  CustomerCount  TotalRevenue  RevenueShare(%)
0  Frequent Low Spender            730   1447660.070            16.25
1            High Value            434   5469382.460            61.38
2                 Other           3175   1994365.374            22.38


In [ ]:
from google.colab import files

# Save the combined summary to CSV
output_file = "Segment_Summary.csv"
combined_summary.to_csv(output_file, index=False)

# Download to your local machine
files.download(output_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# --- Step 1: Run the SQL query for Customer-Month Revenue ---
query_cust_month = """
SELECT
    CustomerID,
    strftime('%Y-%m', InvoiceDate) AS Month,
    SUM(Revenue) AS MonthlyRevenue
FROM transactions
GROUP BY CustomerID, strftime('%Y-%m', InvoiceDate)
ORDER BY CustomerID, Month;
"""

cust_month = pd.read_sql(query_cust_month, conn)

# --- Step 2: Merge with Segmentation (to include Segment info) ---
cust_month_seg = cust_month.merge(segmentation[['CustomerID','Segment']],
                                  on="CustomerID", how="left")

# --- Step 3: Save to CSV ---
output_file = "Customer_Month_Segment.csv"
cust_month_seg.to_csv(output_file, index=False)

# --- Step 4: Download the file ---
from google.colab import files
files.download(output_file)

print("✅ File generated & ready for download:", output_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ File generated & ready for download: Customer_Month_Segment.csv
